# Load and use model
## Luis Arturo
### A01703572

In [1]:
import torch
import torch.nn as nn
from IPython.display import clear_output
import re

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [4]:
class GameRatingModel(nn.Module):
    def __init__(self, vocab_size, embedding_dim, max_length, num_features, num_classes):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.flat_embed_size = max_length * embedding_dim
        
        self.numeric_layer = nn.Sequential(
            nn.Linear(num_features, 64),
            nn.ReLU(),
            nn.BatchNorm1d(64)
        )
        
        self.combined_layers = nn.Sequential(
            nn.Linear(self.flat_embed_size + 64, 128),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.BatchNorm1d(128),
            
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.BatchNorm1d(64),
            
            nn.Linear(64, num_classes)
        )
        
    def forward(self, sequences, features):
        embedded = self.embedding(sequences)
        flat_embedded = embedded.view(embedded.size(0), -1)
        numeric_out = self.numeric_layer(features)
        combined = torch.cat((flat_embedded, numeric_out), dim=1)
        return self.combined_layers(combined)

In [5]:
model = GameRatingModel(
    vocab_size=10000,
    embedding_dim=32,
    max_length=10,
    num_features=14,
    num_classes=9
).to(device)

model.load_state_dict(torch.load('models/first-model.pth'))
model.eval()

# 3. Definir las clases de rating
rating_classes = [
    'Negative',
    'Mixed',
    'Mostly Negative',
    'Mostly Positive',
    'Overwhelmingly Negative',
    'Overwhelmingly Positive',
    'Positive',
    'Very Negative',
    'Very Positive'
]

/tmp/ipykernel_19026/3050004469.py:9: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('models/first-model.pth'))


In [6]:
def clean_title(title):
    title = title.lower()
    title = re.sub(r'[^\w\s]', ' ', title)
    return ' '.join(title.split())

In [7]:
def predict_game_rating(title, price, win, mac, linux, steam_deck, days_market, avg_hours, avg_helpful):
    # Preparar inputs
    sequence = torch.zeros((1, 10), dtype=torch.long).to(device)  # Dummy sequence
    
    features = torch.tensor([[
        price, price, 0,  # price_final, price_original, discount
        win, mac, linux, steam_deck,
        days_market,
        avg_hours, avg_hours/2, avg_hours/2,  # avg_hours, median_hours, std_hours
        0.5, avg_helpful, 0  # recommendation_ratio, avg_helpful, avg_funny
    ]], dtype=torch.float32).to(device)
    
    # Hacer predicción
    with torch.no_grad():
        outputs = model(sequence, features)
        probabilities = torch.softmax(outputs, dim=1)
        _, predicted = torch.max(outputs, 1)
    
    # Retornar resultados
    return rating_classes[predicted.item()], {rating_classes[i]: prob.item() for i, prob in enumerate(probabilities[0])}


In [8]:
def predict_interactive():
    print("Enter game information:")
    title = input("Game title: ")
    price = float(input("Price (USD): "))
    win = int(input("Windows compatible? (1/0): "))
    mac = int(input("Mac compatible? (1/0): "))
    linux = int(input("Linux compatible? (1/0): "))
    steam_deck = int(input("Steam Deck compatible? (1/0): "))
    days_market = int(input("Days in market: "))
    avg_hours = float(input("Average play hours: "))
    avg_helpful = float(input("Average helpful votes: "))
    
    rating, probs = predict_game_rating(
        title, price, win, mac, linux, 
        steam_deck, days_market, avg_hours, avg_helpful
    )
    
    clear_output()
    print(f"\nPrediction for '{title}':")
    print(f"Predicted rating: {rating}")
    print("\nProbabilities for each rating:")
    for rating, prob in sorted(probs.items(), key=lambda x: x[1], reverse=True):
        print(f"{rating}: {prob*100:.2f}%")

In [9]:
while True:
    predict_interactive()
    if input("\nPredict another game? (y/n): ").lower() != 'y':
        break


Prediction for 'I like pupsicles':
Predicted rating: Mostly Positive

Probabilities for each rating:
Mostly Positive: 100.00%
Negative: 0.00%
Mixed: 0.00%
Mostly Negative: 0.00%
Overwhelmingly Negative: 0.00%
Overwhelmingly Positive: 0.00%
Positive: 0.00%
Very Negative: 0.00%
Very Positive: 0.00%
